# Project 1: Customer Database
**This is the first of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
Your team has been hired by the company X as data scientists. X makes gadgets for a wide range of industrial and commercial clients.

As in-house data scientists, your teams first task, as per request from your new boss, is to optimize business operations. You have decided that a good first step would be to analyze the companys historical sales data to gain a better understanding of where profit is coming from. It may also reveal some low hanging fruit in terms of business opportunities.

To get started, you have called the IT department to get access to the customer and sales transactions database. To your horror you've been told that such a database doens't exist, and the only record of sales transactions is kept by John from finance in an Excel spreadsheet. So you've emailed John asking for a CSV dump of the spreadsheet...

In this project you need to clean the data you got from John, enrich it with further data, prepare a database for the data, and do some data analysis. The project is comprised of five parts. They are intended to be solved in the order they appear, but it is highly recommended that you read through all of them and devise an overall strategy before you start implementing anything.

## Part 1: Cleaning the data
John has emailed you the following link to the CSV dump you requested.

- [transactions.csv](https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv)

It seems as though he has been a bit sloppy when keeping the records. 

In this part you should:
- Explain what the data is
- Clean it to prepare it for inserting into a database and doing data analysis 

---
### Solution

We start off by fetching the data, checking the data types and the first few lines to get a general feeling for what we are working with.

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('transactions.csv')

print(df.dtypes)

df.head(10)

part       object
company    object
country    object
city       object
price      object
date       object
dtype: object


,part,company,country,city,price,date
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05
1,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26
2,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30
3,76117-001,Twitterbeat,France,Annecy,1075.82€,2016-01-02 02:32:30
4,44946-1046,Chatterbridge,Spain,Barcelona,412.55€,2016-01-02 04:51:55
5,16729-167,Chatterbridge,Spain,Barcelona,359.52€,2016-01-02 07:20:59
6,52125-444,Voomm,France,Paris,266.62€,2016-01-02 07:40:37
7,43419-018,Buzzbean,Germany,Düsseldorf,103.45€,2016-01-02 08:57:57
8,54092-515,Zooxo,United Kingdom,London,£704.94,2016-01-02 09:09:01
9,24286-1562,Lajo,Greece,Thessaloniki,317.65€,2016-01-02 11:01:32


The data consists of:
- Part serial number
- Company name
- Company location, country
- Company location, city
- Price of the part
- Date of transaction

Apparently there are some typos in a couple of dates that need to be fixed before they can be converted to datetime format:

In [ ]:
# Skip this cell unless you want to see the try-except in action
for i in range(len(df)):
    try:
        pd.to_datetime(df.date[i])
    except ValueError:
        raise ValueError("Incorrect data format: ", df.date[i], ' row: ', i)

In [6]:
# ValueError: ('Incorrect data format: ', '2016-06-32 07:22:28', ' row: ', 3539)
# ValueError: ('Incorrect data format: ', '2016-06-32 08:08:48', ' row: ', 3540)
df.date[3539] = '2016-06-30 07:22:28'
df.date[3540] = '2016-06-30 08:08:48'

We then check how many unique values we have in each column and whether there are any missing values or typos. We also put date into a proper datetime format in order to get earliest and latest date.

In [7]:
print('number of rows: ', len(df), '\n')
print('unique parts: ', len(df.part.unique()))
print('nan: ', df.part.isna().sum(), '\n')

print(df.country.unique())
print('nan: ', df.country.isna().sum(), '\n')

print(df.city.unique())
print('nan: ', df.city.isna().sum(), '\n')

print(df.company.unique())
print('company nan: ', df.company.isna().sum(), '\n')

print('price nan: ', df.price.isna().sum())
print('date nan: ', df.date.isna().sum(), '\n')

print('price in €: ', len(df[df['price'].str.contains("€")==True]))
print('price in £: ', len(df[df['price'].str.contains("£")==True]))
print('price in $: ', len(df[df['price'].str.contains("$")==True]))
print('price in ¥: ', len(df[df['price'].str.contains("¥")==True]))
print('undefined price: ', len(df[df['price'].str.contains("[£$¥€]")==False]), '\n')

df['datetime'] = pd.to_datetime(df.date)
print('date min: ', df.datetime.min())
print('date max: ', df.datetime.max())
df.drop(['date'], inplace=True, axis=1)

df[df['price'].str.contains("[£$¥€]")==False]

number of rows:  20568 

unique parts:  101
nan:  10 

['Spain' 'Greece' 'France' 'Germany' 'United Kingdom' 'Portugal'
 'United States' 'Netherlands' 'Japan' 'Switzerland' nan 'US' 'Tyskland'
 'Portuga']
nan:  2171 

['Barcelona' 'Thessaloniki' 'Athens' 'Annecy' 'Paris' 'Düsseldorf'
 'London' 'Braga' 'Nanterre' 'Amadora\t' 'New York' 'Arnhem' 'Nice'
 'Lisbon' 'Amsterdam' 'Porto' 'Boston' 'Niihama' 'Almada' 'Aranhas'
 'Heraklion' 'Amiens' 'Patras' 'Arcueil' 'Lyon' 'Asaka' 'Champagnole'
 'Zürich' nan 'Monção' 'Vila Fria']
nan:  33 

['Chatterbridge' 'Lajo' 'Flipstorm' 'Twitterbeat' 'Voomm' 'Buzzbean'
 'Zooxo' 'Brainsphere' 'Thoughtmix' 'Wordify' 'Teklist' 'Avaveo' 'Ntags'
 'Innojam' 'Shufflebeat' 'Zoonder' 'Kanoodle' 'Gabcube' 'Roodel'
 'Riffpath' 'Eimbee' 'Yozio' 'Rhycero' 'Realpoint' 'Gabtune' 'Bubblemix'
 'Gevee' 'Tagtune' 'Zooxo.' 'Laj0' 'Ntagz' ' -' ' a' 'aa' 'Thoughtmixz']
company nan:  0 

price nan:  1
date nan:  0 

price in €:  17275
price in £:  1672
price in $:  20567
price 

,part,company,country,city,price,datetime
2414,61398-0828,Voomm,France,Paris,465.6,2016-04-15 06:51:38
2415,36987-1697,Flipstorm,Greece,Athens,1266.68,2016-04-15 08:27:18
2526,0603-6134,Buzzbean,Germany,Düsseldorf,829.3,2016-04-21 02:54:36
2528,52380-1102,Teklist,NaN,Arnhem,357.78,2016-04-21 04:07:31
8683,49349-820,Chatterbridge,Spain,Barcelona,-,2017-03-03 15:24:39
8684,10267-2529,Thoughtmix,Portugal,Amadora\t,-,2017-03-03 18:07:56
8685,13537-259,Ntags,Portugal,Lisbon,-,2017-03-03 19:08:54
8934,68084-172,Twitterbeat,France,Annecy,void,2017-03-18 01:53:38
10328,54868-0823,Chatterbridge,Spain,Barcelona,void,2017-06-02 06:49:09
10329,41163-428,Avaveo,France,Nice,-,2017-06-02 07:04:35


#### Issues with data:
- Missing values in part, country, city and price **✔**-ish (What should we do with part?)
- Typos in country and city **✔**
- Germany/Tyskland, United States/US, Portugal/Portuga **✔**
- Typos in date (already fixed) **✔**
- Prices are in different currencies **✔**
- Some prices unidentified **✔**-ish (Some negative prices)
- Typos and nonsense company names **✔**

#### Cleanup:
Lets start with fixing typos in company, country and city. The correct currency for four rows can be deduced from country/city. Rows with missing company and/or price are removed since without that the data is of very limited use.

In [8]:
dfc = df # df cleaned

dfc['company'] = df['company'].replace({'Laj0':'Lajo','Zooxo.':'Zooxo','Thoughtmixz':'Thoughtmix'})
dfc['country'] = df['country'].replace({'Tyskland':'Germany', 'US':'United States', 'Portuga':'Portugal'})
dfc['city'] = df['city'].replace({'Amadora\t':'Amadora'})

dfc = dfc[dfc.company != ' -']
dfc = dfc[dfc.company != ' a']
dfc = dfc[dfc.company != 'aa']

dfc = dfc[dfc.price != '-']
dfc = dfc[dfc.price != 'void']
dfc = dfc[dfc.price != 'na']
dfc = dfc[dfc.price.isna()==False]

dfc.iloc[[2414],[4]] = dfc.iloc[[2414],[4]] + '€'
dfc.iloc[[2415],[4]] = dfc.iloc[[2415],[4]] + '€'
dfc.iloc[[2526],[4]] = dfc.iloc[[2526],[4]] + '€'
dfc.iloc[[2528],[4]] = dfc.iloc[[2528],[4]] + '€'

# dfc[dfc['price'].str.contains("[£$¥€]")==False]

The missing city values are recovered by comparing rows with the same company and the missing country values are likewise deduced from the city.

In [9]:
dfc['city'] = dfc['city'].fillna(dfc['company'].map({
    'Brainsphere':'Braga', 
    'Shufflebeat':'Porto',
    'Ntags':'Lisbon',
    'Thoughtmix':'Amadora',
    'Yozio':'Patras',
    'Twitterbeat':'Annecy',
    'Zooxo':'London',
    'Zoonder':'Boston',
    'Teklist':'Arnhem',
    'Wordify':'New York',
    'Kanoodle':'Niihama' 
}))

dfc['country'] = dfc['country'].fillna(dfc['city'].map({
    'Arnhem':'Netherlands',
    'Braga':'Portugal',
    'Düsseldorf':'Germany',
    'Annecy':'France',
    'Heraklion':'Greece',
    'Barcelona':'Spain',
    'Lisbon':'Portugal',
    'Athens':'Greece',
    'Amadora':'Portugal',
    'Aranhas':'Portugal',
    'New York':'United States',
    'Patras':'Greece',
    'Amiens':'France',
    'Almada':'Portugal',
    'Boston':'United States',
    'Porto':'Portugal',
    'Paris':'France',
    'Nice':'France',
    'London':'United Kingdom',
    'Niihama':'Japan',
    'Asaka':'Japan',
    'Nanterre':'France',
    'Arcueil':'France',
    'Lyon':'France',
    'Thessaloniki':'Greece',
    'Amsterdam':'Netherlands',
    'Champagnole':'France',
    'Zürich':'Germany'
}))

The price is then extracted and placed in a new column corresponding to the currency.

In [10]:
dfc['eur'] = np.where(dfc['price'].str.contains("€")==True, dfc['price'].map(lambda x: x.lstrip('€').rstrip('€')), None)
dfc['usd'] = np.where(dfc['price'].str.contains("\$")==True, dfc['price'].map(lambda x: x.lstrip('$').rstrip('$')), None)
dfc['gbp'] = np.where(dfc['price'].str.contains("£")==True, dfc['price'].map(lambda x: x.lstrip('£').rstrip('£')), None)
dfc['yen'] = np.where(dfc['price'].str.contains("¥")==True, dfc['price'].map(lambda x: x.lstrip('¥').rstrip('¥')), None)

dfc['eur'] = dfc['eur'].astype(float)
dfc['usd'] = dfc['usd'].astype(float)
dfc['gbp'] = dfc['gbp'].astype(float)
dfc['yen'] = dfc['yen'].astype(float)

#dfc.head(20)

In [11]:
#print(dfc.part.unique(), '\n')
print(dfc.company.unique(), '\n')
print(dfc.country.unique(), '\n')
print(dfc.city.unique(), '\n')

print(len(dfc[dfc.part.isna()==True]))
print(len(dfc[dfc.company.isna()==True]))
print(len(dfc[dfc.country.isna()==True]))
print(len(dfc[dfc.city.isna()==True]))
print(len(dfc[dfc.price.isna()==True]))
print(len(dfc[dfc.datetime.isna()==True]))

#dfc.head(20)

['Chatterbridge' 'Lajo' 'Flipstorm' 'Twitterbeat' 'Voomm' 'Buzzbean'
 'Zooxo' 'Brainsphere' 'Thoughtmix' 'Wordify' 'Teklist' 'Avaveo' 'Ntags'
 'Innojam' 'Shufflebeat' 'Zoonder' 'Kanoodle' 'Gabcube' 'Roodel'
 'Riffpath' 'Eimbee' 'Yozio' 'Rhycero' 'Realpoint' 'Gabtune' 'Bubblemix'
 'Gevee' 'Tagtune' 'Ntagz'] 

['Spain' 'Greece' 'France' 'Germany' 'United Kingdom' 'Portugal'
 'United States' 'Netherlands' 'Japan' 'Switzerland'] 

['Barcelona' 'Thessaloniki' 'Athens' 'Annecy' 'Paris' 'Düsseldorf'
 'London' 'Braga' 'Nanterre' 'Amadora' 'New York' 'Arnhem' 'Nice' 'Lisbon'
 'Amsterdam' 'Porto' 'Boston' 'Niihama' 'Almada' 'Aranhas' 'Heraklion'
 'Amiens' 'Patras' 'Arcueil' 'Lyon' 'Asaka' 'Champagnole' 'Zürich'
 'Monção' 'Vila Fria'] 

10
0
0
0
0
0


We then add ID columns for part, company, country and city in preperation for the database.

In [12]:
dfc_part_s = dfc.sort_values(by=['part'])
part_u = dfc_part_s.part.unique()
df_part = pd.DataFrame({'part_id':list(range(1, len(part_u)+1)), 'name':part_u })

dfc_company_s = dfc.sort_values(by=['company'])
company_u = dfc_company_s.company.unique()
df_company = pd.DataFrame({'company_id':list(range(1, len(company_u)+1)), 'name':company_u })

dfc_country_s = dfc.sort_values(by=['country'])
country_u = dfc_country_s.country.unique()
df_country = pd.DataFrame({'country_id':list(range(1, len(country_u)+1)), 'name':country_u })

dfc_city_s = dfc.sort_values(by=['city'])
city_u = dfc_city_s.city.unique()
df_city = pd.DataFrame({'city_id':list(range(1, len(city_u)+1)), 'name':city_u })

dfm = dfc # df merged

dfm = (dfm.merge(df_part, left_on='part', right_on='name')).drop(['name'], axis=1)
dfm = (dfm.merge(df_company, left_on='company', right_on='name')).drop(['name'], axis=1)
dfm = (dfm.merge(df_country, left_on='country', right_on='name')).drop(['name'], axis=1)
dfm = (dfm.merge(df_city, left_on='city', right_on='name')).drop(['name'], axis=1)

In [13]:
print(df_country)
dfm.sort_values(by=['datetime']).head()

   country_id            name
0           1          France
1           2         Germany
2           3          Greece
3           4           Japan
4           5     Netherlands
5           6        Portugal
6           7           Spain
7           8     Switzerland
8           9  United Kingdom
9          10   United States


,part,company,country,city,price,datetime,eur,usd,gbp,yen,part_id,company_id,country_id,city_id
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05,784.79,NaN,NaN,NaN,66,5,7,11
7402,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26,187.99,NaN,NaN,NaN,79,13,3,28
5951,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30,221.73,NaN,NaN,NaN,23,7,3,10
2765,76117-001,Twitterbeat,France,Annecy,1075.82€,2016-01-02 02:32:30,1075.82,NaN,NaN,NaN,96,24,1,5
63,44946-1046,Chatterbridge,Spain,Barcelona,412.55€,2016-01-02 04:51:55,412.55,NaN,NaN,NaN,33,5,7,11


---
## Part 2: Enriching the data

A common task for a data scientists is to combine or enrich data from internal sources with data available from external sources. The purpose of this can be either to fix issues with the data or to make it easier to derive insights from the data.

In this part you should enrich your data with data from at least one external source. You may look to part 4 for some  inspiration as to what is required. Your solution should be automated, i.e., you can not ask the reader of your notebook to download any data manually. You should argue why and what you expect to achieve by the enrichments you are doing.

---
### Solution

In [ ]:
# We should probably have a column for each currency and fill in the gaps using exchange rate info from the api

---
## Part 3: Creating a database
Storing data in a relational database has the advantages that it is persistent, fast to query, and it will be easier access for other employees at Weyland-Yutani.

In this part you should:
- Create a database and table(s) for the data
- Insert data into the tables

You may use SQLite locally to do this. You should argue why you choose to store your data the way you do. 

---
### Solution

The database is the foundation of the data and needs to be constructed so that consistency and integrity is maintained. We decided to put part, company, country and city in separate reference tables and connect them to the main table trans* with foreign keys. The most notable benefits of this are:

1. Less risk of inconsistencies between rows because the data is present in only one place
2. Less risk of incorrect data being inserted into the database because it throws an error if the correct ID does not exist in the reference tables

Pandas does not seem to include a way to insert dataframes directly into existing tables and insists on creating its own using the to_sql function. To bypass this, we simply use those tables temporarily and move the data to our own.

*\*It is an established convention that table names are singlar. Transaction, however, is a reserved word in SQL, hence the stubby trans.*

In [146]:
import sqlalchemy
from sqlalchemy import create_engine

engine = sqlalchemy.create_engine('sqlite:///project1.sqlite', echo=False)

df_part.to_sql('part_temp', con=engine, index=False, if_exists='replace')
df_company.to_sql('company_temp', con=engine, index=False, if_exists='replace')
df_country.to_sql('country_temp', con=engine, index=False, if_exists='replace')
df_city.to_sql('city_temp', con=engine, index=False, if_exists='replace')
dfm.to_sql('trans_temp', con=engine, index=True, if_exists='replace')

In [147]:
import sqlite3
conn = sqlite3.connect('project1.sqlite')
c = conn.cursor()

c.execute('DROP TABLE IF EXISTS trans')
conn.commit()
c.execute('DROP TABLE IF EXISTS part')
conn.commit()
c.execute('DROP TABLE IF EXISTS company')
conn.commit()
c.execute('DROP TABLE IF EXISTS city')
conn.commit()
c.execute('DROP TABLE IF EXISTS country')
conn.commit()
conn.close()

NOT NULL restrictions are included to further reduce the risk of incorrect data.

In [148]:
import sqlite3
conn = sqlite3.connect('project1.sqlite')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS part(id INTEGER PRIMARY KEY, name VARCHAR(50) /*NOT NULL*/)')
conn.commit()

c.execute('CREATE TABLE IF NOT EXISTS company(id INTEGER PRIMARY KEY, name VARCHAR(50) NOT NULL)')
conn.commit()

c.execute('CREATE TABLE IF NOT EXISTS country(id INTEGER PRIMARY KEY, name VARCHAR(50) NOT NULL)')
conn.commit()

c.execute('CREATE TABLE IF NOT EXISTS city(id INTEGER PRIMARY KEY, name VARCHAR(50) NOT NULL)')
conn.commit()

query = '''
CREATE TABLE IF NOT EXISTS trans(
    id INTEGER PRIMARY KEY,
    part_id INTEGER,
    company_id INTEGER,
    country_id INTEGER,
    city_id INTEGER,
    eur DECIMAL(13, 4) /*NOT NULL*/,
    usd DECIMAL(13, 4) /*NOT NULL*/,
    gbp DECIMAL(13, 4) /*NOT NULL*/,
    yen DECIMAL(13, 4) /*NOT NULL*/,
    transaction_date DATETIME,
    FOREIGN KEY(part_id) REFERENCES part(id),
    FOREIGN KEY(city_id) REFERENCES city(id),
    FOREIGN KEY(country_id) REFERENCES country(id)
)
'''
c.execute(query)
conn.commit()

In [149]:
c.execute('INSERT INTO part (id, name) SELECT part_id, name FROM part_temp')
conn.commit()

c.execute('INSERT INTO company (id, name) SELECT company_id, name FROM company_temp')
conn.commit()

c.execute('INSERT INTO country (id, name) SELECT country_id, name FROM country_temp')
conn.commit()

c.execute('INSERT INTO city (id, name) SELECT city_id, name FROM city_temp')
conn.commit()

query = '''
INSERT INTO trans (part_id, company_id, country_id, city_id, eur, usd, gbp, yen, transaction_date)
SELECT part_id, company_id, country_id, city_id, eur, usd, gbp, yen, datetime FROM trans_temp
'''
c.execute(query)
conn.commit()

c.execute('DROP TABLE IF EXISTS trans_temp')
conn.commit()
c.execute('DROP TABLE IF EXISTS part_temp')
conn.commit()
c.execute('DROP TABLE IF EXISTS company_temp')
conn.commit()
c.execute('DROP TABLE IF EXISTS city_temp')
conn.commit()
c.execute('DROP TABLE IF EXISTS country_temp')
conn.commit()

c.execute('PRAGMA foreign_keys = TRUE') # We actually need to activate foreign key enforcement
conn.commit()

In [150]:
pd.read_sql_query('SELECT * FROM trans where eur is null order by transaction_date desc limit 5', conn)

,id,part_id,company_id,country_id,city_id,eur,usd,gbp,yen,transaction_date
0,17175,43,28,10,12,None,476.66,NaN,None,2018-12-31 02:24:27.000000
1,17687,52,29,9,18,None,NaN,738.70,None,2018-12-30 23:22:57.000000
2,16497,8,26,10,22,None,438.62,NaN,None,2018-12-30 22:11:40.000000
3,18404,27,29,9,18,None,NaN,195.75,None,2018-12-30 08:14:21.000000
4,17191,37,28,10,12,None,193.15,NaN,None,2018-12-29 10:41:40.000000


In [ ]:
# Skip this cell unless you want to see the foreign key enforcement in action
query = '''
INSERT INTO trans (part_id, company_id, country_id, city_id, eur, usd, gbp, yen, transaction_date)
VALUES (66, 5, 12345678, 11, 784.79, NULL, NULL, NULL, '2016-01-02 00:01:05.000000')
'''

#query = '''
#INSERT INTO country (id, name)
#VALUES (12345678, 'Imagination land')
#'''

c.execute(query)
conn.commit()

When the connection is closed explicitly by code or implicitly by program exit then any outstanding transaction is rolled back. (The rollback is actually done by the next program to open the database.) If there is no outstanding transaction open then nothing happens.


In [152]:
conn.close()

---
## Part 4: Analyzing the data
You are now ready to analyze the data. Your goal is to gain some actionable business insights to present to your boss. 

In this part, you should ask some questions and try to answer them based on the data. You should write SQL queries to retrieve the data. For each question, you should state why it is relevant and what you expect to find.

To get you started, you should prepare answers to the following questions. You should add more questions.
#### Who are the most profitable clients?
Knowing which clients that generate the most revenue for the company will assist your boss in distributing customer service ressources.

#### Are there any clients for which profit is declining?
Declining profit from a specific client may indicate that the client is disatisfied with the product. Gaining a new client is often much more work than retaining one. Early warnings about declining profit may help your boss fighting customer churn.


Remember, you are taking this to your new boss, so think about how you present the data.

---
### Solution

In [122]:
conn = sqlite3.connect('project1.sqlite')

query = '''
SELECT
    c.name,
    t.eur,
    t.usd as 'dolla dolla bills y''all',
    t.gbp,
    t.yen,
    t.transaction_date
FROM trans t
JOIN company c on t.company_id = c.id
order by eur desc
limit 5
'''
pd.read_sql_query(query, conn)

,name,eur,dolla dolla bills y'all,gbp,yen,transaction_date
0,Thoughtmix,2174.74,None,None,None,2018-04-06 06:10:25.000000
1,Ntags,2140.64,None,None,None,2016-09-09 04:47:05.000000
2,Ntags,2126.82,None,None,None,2016-07-29 01:30:35.000000
3,Flipstorm,2092.49,None,None,None,2016-09-07 13:31:12.000000
4,Twitterbeat,2035.70,None,None,None,2018-11-26 02:42:16.000000


---
## Part 5: Performance
Your boss is very impressed with what you have achieved in less than two weeks, and he would like to take your idea of storing the customer and sales data in a relational database to production. However, John is concerned that the solution will not scale. His experience is telling him that you will see many occurrences of the following queries.

- Show all sales to company X between time $t_1$ and time $t_2$
- Show the latest X sales in the database
- Show total sales per company per day

Show that Johns concern is not justified.

---
### Solution

In [ ]:
# Concerns of a massive and therefore slow a database? Fix with indexes?
# Or is this just a matter of making some gigantic query and showing that it is fast enough?

---